In [3]:
import os
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from AudioDataset import AudioDataset
from CLassicalML import Preprocessor, RandomForestModel, XGBoostModel, LightGBMModel, LogisticRegressionModel, \
    ModelComparer
from DescriptiveStatistics import DescriptiveStatistics
from HyperParameterTuning import HyperParameterTuning
import torch.nn as nn
import torch.optim as optim

APPLY_DESCRIPTIVE_STATISTICS = False
APPLY_CLASSICAL_ML = True
APPLY_LSTM = False

## Load the data

In [4]:
train_labels = pd.read_csv('compare22-KSF/lab/train.csv').sort_values(by='filename')
test_labels = pd.read_csv('compare22-KSF/lab/test.csv').sort_values(by='filename')
devel_labels = pd.read_csv('compare22-KSF/lab/devel.csv').sort_values(by='filename')
# convert the test labels to nan
test_labels['label'] = float('nan')

directory = 'compare22-KSF/wav'
all_files = os.listdir(directory)
train_wav_files = sorted([os.path.join(directory, file) for file in all_files if file.startswith('train')])
devel_wav_files = sorted([os.path.join(directory, file) for file in all_files if file.startswith('devel')])
test_wav_files = sorted([os.path.join(directory, file) for file in all_files if file.startswith('test')])

In [5]:
# Create a label map dictionary for mapping the string labels to integers
label_map = {
    'Prolongation': 0,
    'no_disfluencies': 1,
    'Fillers': 2,
    'Block': 3,
    'Modified': 4,
    'SoundRepetition': 5,
    'WordRepetition': 6,
    'Garbage': 7
}

train_labels['label'] = train_labels['label'].map(label_map)
devel_labels['label'] = devel_labels['label'].map(label_map)

## Descriptive Statistics

In [6]:
if APPLY_DESCRIPTIVE_STATISTICS:
    # Create an instance of the DescriptiveStatistics class and run the statistical tests and plots
    ds = DescriptiveStatistics(train_wav_files, test_wav_files, devel_wav_files)
    ds.run()

## Classical Machine Learning

In [ ]:
if APPLY_CLASSICAL_ML:
    # Collect the features for the train and devel datasets
    methods = ['tempo_and_beats', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast',
               'spectral_flatness', 'spectral_rolloff', 'zero_crossing_rate', 'rms_energy']
    pre_ps = Preprocessor(methods)
    # apply feature extraction methods to the audio files
    train_df = pre_ps.run(list_of_audio_files=train_wav_files, y=train_labels)
    devel_df = pre_ps.run(list_of_audio_files=devel_wav_files, y=devel_labels)
    # Split the data into features and target
    X_train = train_df.drop('label', axis=1)
    y_train = train_df['label']
    X_test = devel_df.drop('label', axis=1)
    y_test = devel_df['label']
    # apply standard scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Create a list of models
    models = [RandomForestModel(), XGBoostModel(), LightGBMModel(), LogisticRegressionModel()]
    # Compare models
    comparer = ModelComparer(models)
    results = comparer.tune_and_evaluate(X_train, y_train, X_test, y_test)
    print(results)


## LSTM

In [2]:
if APPLY_LSTM:
    # Create an instance of the AudioDataset class
    train_dataset = AudioDataset(train_wav_files, train_labels['label'].values.tolist())
    test_dataset = AudioDataset(test_wav_files, test_labels['label'].values.tolist())
    devel_dataset = AudioDataset(devel_wav_files, devel_labels['label'].values.tolist())
    
    # Create a DataLoader instance to load batches of data
    train_data_loader = DataLoader(train_dataset, batch_size=32)
    test_data_loader = DataLoader(test_dataset, batch_size=32)
    devel_data_loader = DataLoader(devel_dataset, batch_size=32)
    
    param_grid = {
        'input_size': [48000],
        'hidden_size': [32, 64, 128],
        'num_layers': [2, 4, 6],
        'num_classes': [train_labels['label'].nunique()],
        'criterion': [nn.CrossEntropyLoss()],
        'learning_rate': [0.001, 0.01],
        'optimizer': [optim.Adam, optim.SGD],
        'num_epochs': [10, 15, 20, 25, 30]
    }
    
    hpt = HyperParameterTuning(param_grid=param_grid)
    hpt.fit(train_data_loader, devel_data_loader)
    hpt.print_best_model()

NameError: name 'APPLY_LSTM' is not defined